In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64
np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.f1 = nn.Linear(28*28, 50)
        self.f2 = nn.Linear(50, 50)
        self.f3 = nn.Linear(50, 50)
        self.out = nn.Linear(50, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = self.out(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (f1): Linear(in_features=784, out_features=50, bias=True)
    (f2): Linear(in_features=50, out_features=50, bias=True)
    (f3): Linear(in_features=50, out_features=50, bias=True)
    (out): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [2]:
def train_model(model, num_epochs, learning_rate=0.1, momentum=0.95):
    entropy_loss = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    
    for epoch in range(num_epochs):
        model.train() 
        for (data, target) in train_loader:
            data = data.to(device)
            target = target.to(device)
            
            optimizer.zero_grad()           
            outputs = model(data)           
            loss = entropy_loss(outputs, target)
            loss.backward()                 
            optimizer.step()                

In [4]:
train_model(model, 20)
torch.save(model.state_dict(), 'weights.pt')

In [66]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            outputs = model(data) 
            _, predicted = torch.max(outputs, 1) 
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100.0 * correct / total
    print(f'Model Accuracy on MNIST Test Set: {accuracy:.2f}%')
    return accuracy

accuracy = evaluate_model(model, test_loader, device)

Model Accuracy on MNIST Test Set: 8.52%


In [63]:
def interval_analysis(model, x, eps):
    f1_weight = model[1].f1.weight.data 
    f1_bias = model[1].f1.bias.data   
    
    f2_weight = model[1].f2.weight.data
    f2_bias = model[1].f2.bias.data  
    
    f3_weight = model[1].f3.weight.data 
    f3_bias = model[1].f3.bias.data 
    
    out_weight = model[1].out.weight.data 
    out_bias = model[1].out.bias.data 
    
    x_normalized = (x - 0.1307) / 0.3081
    
    x_normalized = x_normalized.view(-1)
    x_lower = x_normalized - eps
    x_upper = x_normalized + eps

    f1_lower = torch.zeros(50)
    f1_upper = torch.zeros(50)
    
    for i in range(f1_weight.size(0)):
        sum_lower = 0.0
        sum_upper = 0.0
        for j in range(f1_weight.size(1)): 
            w = f1_weight[i][j].item()
            if w >=0:
                sum_lower += w * x_lower[j]
                sum_upper += w * x_upper[j]
            else:
                sum_lower += w * x_upper[j]
                sum_upper += w * x_lower[j]
        f1_lower[i] = sum_lower + f1_bias[i]
        f1_upper[i] = sum_upper + f1_bias[i]

    f1_lower = torch.clamp(f1_lower, min=0.0)
    f1_upper = torch.clamp(f1_upper, min=0.0)

    f2_lower = torch.zeros(50)
    f2_upper = torch.zeros(50)
    
    for i in range(f2_weight.size(0)):
        sum_lower = 0.0
        sum_upper = 0.0
        for j in range(f2_weight.size(1)):
            w = f2_weight[i][j].item()
            if w >=0:
                sum_lower += w * f1_lower[j]
                sum_upper += w * f1_upper[j]
            else:
                sum_lower += w * f1_upper[j]
                sum_upper += w * f1_lower[j]
        f2_lower[i] = sum_lower + f2_bias[i]
        f2_upper[i] = sum_upper + f2_bias[i]

    f2_lower = torch.clamp(f2_lower, min=0.0)
    f2_upper = torch.clamp(f2_upper, min=0.0)

    f3_lower = torch.zeros(50)
    f3_upper = torch.zeros(50)
    
    for i in range(f3_weight.size(0)):
        sum_lower = 0.0
        sum_upper = 0.0
        for j in range(f3_weight.size(1)):
            w = f3_weight[i][j].item()
            if w >=0:
                sum_lower += w * f2_lower[j]
                sum_upper += w * f2_upper[j]
            else:
                sum_lower += w * f2_upper[j]
                sum_upper += w * f2_lower[j]
        f3_lower[i] = sum_lower + f3_bias[i]
        f3_upper[i] = sum_upper + f3_bias[i]
    
    f3_lower = torch.clamp(f3_lower, min=0.0)
    f3_upper = torch.clamp(f3_upper, min=0.0)

    out_lower = torch.zeros(10)
    out_upper = torch.zeros(10)
    
    for i in range(out_weight.size(0)):
        sum_lower = 0.0
        sum_upper = 0.0
        for j in range(out_weight.size(1)):
            w = out_weight[i][j].item()
            if w >=0:
                sum_lower += w * f3_lower[j]
                sum_upper += w * f3_upper[j]
            else:
                sum_lower += w * f3_upper[j]
                sum_upper += w * f3_lower[j]
        out_lower[i] = sum_lower + out_bias[i]
        out_upper[i] = sum_upper + out_bias[i]
    print(out_lower)
    print(out_upper)
    return out_lower, out_upper


In [64]:

!pip install tqdm
from tqdm import tqdm
eps = [0.01 * i for i in range(1, 11)]
robust_rates = []

for e in eps:
    robust_count = 0
    process_count = 0
    test_num = 10

    for idx, (data, target) in enumerate(tqdm(test_loader)):
        process_count += 1
        data = data.to(device)
        target = target.item()
        

        f_l, f_u = interval_analysis(model, data, e)
        c_l = f_l[target]

        incorrect_u = f_u
        incorrect_u[target] = -1000

        max_incorrect_u = torch.max(incorrect_u)
        
        if c_l > max_incorrect_u:
            print("correct_lower: ", c_l)
            print("max_incorrect_upper: ", max_incorrect_u)
            robust_count +=1
        if process_count >= test_num:
            break
    robust_rate = robust_count / test_num
    print("robust_rate: ", robust_rate)


Defaulting to user installation because normal site-packages is not writeable


  0%|          | 1/10000 [00:01<3:04:19,  1.11s/it]

tensor([-1.2872, -0.8924, -1.0657, -0.8974, -1.1955, -1.0123, -0.5290, -0.8308,
        -0.9499, -0.7195])
tensor([0.5968, 1.0441, 1.0219, 0.9192, 0.8673, 0.9053, 1.0133, 0.7626, 0.9839,
        1.1293])


  0%|          | 2/10000 [00:02<3:06:24,  1.12s/it]

tensor([-1.2875, -0.8726, -1.0299, -0.9247, -1.1996, -0.9723, -0.5380, -0.7559,
        -0.9268, -0.7370])
tensor([0.6320, 1.0234, 1.0519, 0.8790, 0.8748, 0.8993, 1.0087, 0.7828, 0.9987,
        1.0804])


...

tensor([-9.2102, -7.6832, -7.7579, -6.8370, -8.9400, -8.2301, -5.0335, -6.8132,
        -7.9238, -6.1257])
tensor([6.3339, 8.1773, 9.3145, 7.8093, 7.6018, 7.1095, 7.4455, 6.2196, 7.9502,
        8.7966])


  0%|          | 9/10000 [00:10<3:17:05,  1.18s/it]

tensor([-10.4103,  -8.6966,  -8.7940,  -7.7426, -10.1408,  -9.3419,  -5.7258,
         -7.7890,  -8.9862,  -6.9461])
tensor([ 7.2280,  9.2774, 10.6016,  8.9023,  8.6151,  8.0897,  8.4019,  7.0003,
         9.0049,  9.9736])


  0%|          | 9/10000 [00:11<3:41:07,  1.33s/it]

tensor([-9.8383, -8.1822, -8.2739, -7.2871, -9.5534, -8.8315, -5.3864, -7.3167,
        -8.4712, -6.5469])
tensor([ 6.7792,  8.7585, 10.0035,  8.3976,  8.1169,  7.5803,  7.9439,  6.6198,
         8.4869,  9.4130])
robust_rate:  0.0
